<a href="https://colab.research.google.com/github/DSJR741221/DSJackieR/blob/main/RetoEmpleados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Importa las librerías requeridas
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.decomposition import PCA

#1. Lee el archivo CSV y colócalo en un DataFrame
EmpleadosAttrition = pd.read_csv('/content/sample_data/empleadosRETO.csv')

#2. Elimina las columnas irrelevantes
EmpleadosAttrition.drop(['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours'], axis=1, inplace=True)

#3. Crea una columna llamada Year para obtener el año de contratación a partir de 'HiringDate'
EmpleadosAttrition['Year'] = pd.to_datetime(EmpleadosAttrition['HiringDate']).dt.year

#4. Crea la columna 'YearsAtCompany' calculando los años hasta el año 2018
EmpleadosAttrition['YearsAtCompany'] = 2018 - EmpleadosAttrition['Year']

#5. Limpia 'DistanceFromHome', renombra la columna y convierte a entero
EmpleadosAttrition['DistanceFromHome_km'] = EmpleadosAttrition['DistanceFromHome']
EmpleadosAttrition['DistanceFromHome'] = EmpleadosAttrition['DistanceFromHome'].str.replace(' km', '').astype(int)

#6. Borra las columnas ya innecesarias
EmpleadosAttrition.drop(['Year', 'HiringDate', 'DistanceFromHome_km', 'OverTime'], axis=1, inplace=True)

#7. Calcula el ingreso mensual promedio por departamento y guárdalo en SueldoPromedioDepto
SueldoPromedioDepto = EmpleadosAttrition.groupby('Department')['MonthlyIncome'].mean().reset_index()
SueldoPromedioDepto.rename(columns={'MonthlyIncome': 'SueldoPromedio'}, inplace=True)

#8. Escala 'MonthlyIncome' a valores entre 0 y 1
scaler = MinMaxScaler()
EmpleadosAttrition['MonthlyIncome'] = scaler.fit_transform(EmpleadosAttrition[['MonthlyIncome']])

#9. Convierte variables categóricas a numéricas
categorical_vars = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'Attrition']
for var in categorical_vars:
    le = LabelEncoder()
    EmpleadosAttrition[var] = le.fit_transform(EmpleadosAttrition[var])

#10. Calcula la correlación lineal de cada variable con respecto a 'Attrition'
correlations = EmpleadosAttrition.corr()['Attrition'].abs()

#11. Filtra las variables con correlación mayor o igual a 0.1
selected_vars = correlations[correlations >= 0.1].index
EmpleadosAttritionFinal = EmpleadosAttrition[selected_vars]

#12. Aplica PCA al frame EmpleadosAttritionFinal para reducir la dimensionalidad
pca = PCA(n_components=0.8)  # Explicar el 80% de la varianza
EmpleadosAttritionPCA = pca.fit_transform(EmpleadosAttritionFinal.drop(columns=['Attrition']))

#13. Agrega los componentes principales al DataFrame final
for i in range(EmpleadosAttritionPCA.shape[1]):
    EmpleadosAttritionFinal = EmpleadosAttritionFinal.assign(**{f'C{i}': EmpleadosAttritionPCA[:, i]})

# Muestra un resumen del DataFrame final
# print(EmpleadosAttritionFinal.head())

#14.Guarda el set de datos que has formado y que tienes en EmpleadosAttritionFinal en un archivo CSV llamado EmpleadosAttritionFinal.csv.

#(opcional) Mueve la columna 'Attrition' al final
cols = [col for col in EmpleadosAttritionFinal.columns if col != 'Attrition'] + ['Attrition']
EmpleadosAttritionFinal = EmpleadosAttritionFinal[cols]

# Guarda el DataFrame en un archivo CSV
EmpleadosAttritionFinal.to_csv('/content/sample_data/EmpleadosAttritionFinal.csv', index=False)

# Muestra un resumen del DataFrame final
print(EmpleadosAttritionFinal.head())

   Age  EnvironmentSatisfaction  JobInvolvement  JobLevel  JobSatisfaction  \
0   50                        4               3         4                4   
1   36                        2               3         2                2   
2   21                        2               3         1                2   
3   52                        2               3         3                2   
4   33                        2               3         3                3   

   MaritalStatus  MonthlyIncome  TotalWorkingYears  YearsInCurrentRole  \
0              0       0.864269                 32                   4   
1              0       0.207340                  7                   2   
2              2       0.088062                  1                   0   
3              2       0.497574                 18                   6   
4              1       0.664470                 15                   6   

   YearsAtCompany         C0        C1  Attrition  
0               5  20.247667 -4.20

# New Section